In [ ]:
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.


In [ ]:
from google import genai
from google.genai import types
API_KEY = ""   # replace with your API key
client = genai.Client(api_key=API_KEY)

In [ ]:
EDUCATIONS = """Instruction: As a resume writing instructor, optimize the given "education" section of resume in JSON format.
- Ensure integrity and consistency.
- Retain all provided details of education and degree, including institution names, degrees, dates, GPA.
- Only return "education" section as a response, don't include any other sections.
- Avoid missing or adding any educational details.

<output_example>
"education": [
  {{
    "degree": "Masters of Science - Computer Science (Thesis)",
    "university": "Arizona State University, Tempe, USA",
    "from_date": "Aug 2023",
    "to_date": "May 2025",
    "grade": "3.8/4"
  }},
  {{
    "degree": "Bachelor of Science - Computer Science",
    "university": "Bangalore University, Bangalore, India",
    "from_date": "Aug 2019",
    "to_date": "May 2023",
    "grade": "3.6/4"
  }}
]
</output_example>

"""

PROJECTS = """Instructions: Refine and rewrite the given "projects" section in JSON format.
Follow these criteria:
1. Only return refined "projects" section as a response, don't include other sections.
2. Format each project in your response as the following output template.
    "projects": [
        {{
          "name": "project name1",
          "link": "https://devpost.com/software/project1",
          "from_date": "Nov 2023",
          "to_date": "Nov 2023",
          "description": [
            "introduction of project task, key features, and results."
          ]
        }},
        {{
          "name": "project name2",
          "link": "https://devpost.com/software/project2",
          "from_date": "June 2022",
          "to_date": "July 2022",
          "description": [
            "introduction of project task, key features, and results."
          ]
        }}
    ]

3. Retain at most top 3 relevant projects to the job requirements in the resume.
4. In each project description, it should include info, such as Task, Feature, Result.
5. Improve clarity and alignment with the job requirements.
6. Avoid adding details not given in the original resume data.
"""

SKILLS = """Instructions: Refine and rewrite the given "skill_section" section of resume in JSON format.
Follow these criteria:
1. Only return "skill_section" section as a response, don't include any other sections.
2. Remove irrelevant skills to the job description.
3. Format your response as the following template.
    "skill_section": [
        {{
          "name": "Programming Languages",
          "skills": ["Python", "JavaScript"]
        }},
        {{
          "name": "Cloud Platform",
          "skills": [ "Azure", "AWS"]
        }}
      ]

4. Ensure most relevant skills in resume are retained, and enhance alignment with the job description.
5. Add other relevant skills showed in other sections of resume if they are aligned with job requirements.
6. Avoid adding new skills that are not showed in resume.
"""

EXPERIENCE = """Instructions: Refine and rewrite the given "work_experience" section of resume in JSON format.
Follow these criteria:
1. Return refined "work_experience" section as a response, don't include other sections.
2. Format each experience as the following output template.
    "work_experience": [
        {{
          "role": "Software Engineer",
          "company": "Winjit Technologies",
          "location": "Pune, India"
          "from_date": "Jan 2020",
          "to_date": "Jun 2022",
          "description": [
            "Engineered 10+ RESTful APIs Architecture and Distributed services.",
            "Designed 30+ low-latency responsive UI/UX application features with high-quality web architecture.",
            "Managed and optimized large-scale Databases. (Systems Design)."
          ]
        }}
    ]

3. Retain at most top 3 relevant experience without altering factual details.
4. Avoid adding details not given in the original resume data.
5. In each experience, description should include information about responsibilities and impacts as string text.
6. Ensure clarity, coherence, and alignment with the job description.
"""

SUMMARY = """Instructions: As a resume writing instructor, optimize the given "summary" section of resume in JSON format.
- Retain key details while enhancing clarity, conciseness, and alignment with the job description.
- Ensure a strong, informative summary without adding new, unprovided content.
- Remove irrelevant and redundant content.
- Only return "summary" section as a response, don't include any other sections.
- No more than 100 words.

<output_example>
{{
  "summary": "Results-driven Marketing Professional with 5+ years of experience in digital marketing, brand strategy, and campaign management. Proven track record of increasing online engagement by 40% and driving a 25% boost in sales through data-driven strategies. Skilled in SEO, social media marketing, and analytics tools like Google Analytics and HubSpot. Passionate about creating innovative marketing solutions to help businesses grow. Seeking to leverage expertise in a dynamic, growth-oriented organization."
}}
</output_example>"""

In [ ]:
import json
import time
json_file = "/content/merged_json_ds.json"
with open(json_file, 'r') as file:
    data = json.load(file)

In [ ]:
merged_json = []

In [ ]:
# can select one out of these three sections, comment the rest of sections
section_dict = {
    # 'skill_section': SKILLS
    # 'projects': PROJECTS,
    'work_experience': EXPERIENCE
}


In [ ]:
for k in section_dict:

  for instance in data:
    if len(merged_json) > 150:
      break
    else:
      input = instance.get("input")
      if input.get("resume").get("work_experience") is None or len(input.get("resume").get("work_experience"))<1:
        continue
      response = client.models.generate_content(
          model="gemini-1.5-flash", contents=f"{input}",
          config=types.GenerateContentConfig(
            system_instruction=f"{section_dict[k]}",
            max_output_tokens= 2500,
            top_k= 200,
            top_p= 0.9,
            temperature= 0.8,
            response_mime_type= 'application/json',
            stop_sequences= ['\n']
        )
      )
      # json_response = json.loads(response.text)
      # refined_sections[key] = json_response

      json_output = None
      if response.text.startswith("```json"):
        generated_text = response.text
        generated_text=generated_text.replace("```json", "").replace("```", "").strip()
        json_output = json.loads(generated_text)
      elif isinstance(response.text, str) and response.text.startswith("{"):
        json_output = json.loads(response.text)
        if json_output.get(k) is None or len(json_output.get(k))<1:
          continue
      elif isinstance(response.text, str):
        json_output = json.loads("{"+response.text+"}")
        if json_output.get(k) is None or len(json_output.get(k))<1:
          continue
      elif isinstance(response.text, dict):
        json_output = response.text
      else:
        print("Unknown response type")

      if json_output is not None:
        # print(json.dumps(json_output, indent=4))
        for key in json_output:
          if(len(json_output[key])>0):
            input["instruction"] = section_dict[k]
            full_json = {
                "input": input,
                "output": json_output
            }
            merged_json.append(full_json)
      time.sleep(60)

In [ ]:
print(len(merged_json))

101


In [ ]:
print(response.text)

{"work_experience": [{"role": "Information Technology", "company": "Company Name", "location": "City, State", "from_date": "02/2011", "to_date": "Current", "description": ["Managed accreditation efforts for a major department modernization project, resulting in 3 successful Authorization To Operate (ATO) decisions.", "Expanded responsibilities to include all departmental accreditation efforts, leading to another 3 successful ATOs.", "Currently working on 4 new accreditations, including re-authorization for an existing project. Successfully implemented vulnerability management for existing accredited systems.", "Successfully coordinated with ONI Enterprise on critical design decisions, ensuring smooth program integration into the Enterprise.", "Managed specific STIG compliance and hardening for various configurations and services required for domain integration.", "Analyzed complex computer systems to assess vulnerability and risk. Supervised 5 external computer consultants and vendors.

In [ ]:
refined_resume_section_file = "/content/eval_json_exp_ds.json"
with open(refined_resume_section_file, "w+", encoding='utf-8') as output_file:
  json.dump(merged_json, output_file, indent=4)

In [ ]:
merged_json = []

In [ ]:
# can select one out of these three sections, comment the rest of sections
section_dict = {
    'skill_section': SKILLS
}

In [ ]:
for k in section_dict:

  for instance in data:
    if len(merged_json) > 150:
      break
    else:
      input = instance.get("input")
      if input.get("resume").get("skill_section") is None or len(input.get("resume").get("skill_section"))<1:
        continue
      response = client.models.generate_content(
          model="gemini-1.5-flash", contents=f"{input}",
          config=types.GenerateContentConfig(
            system_instruction=f"{section_dict[k]}",
            max_output_tokens= 2500,
            top_k= 200,
            top_p= 0.9,
            temperature= 0.8,
            response_mime_type= 'application/json',
            stop_sequences= ['\n']
        )
      )
      # json_response = json.loads(response.text)
      # refined_sections[key] = json_response

      json_output = None
      if response.text.startswith("```json"):
        generated_text = response.text
        generated_text=generated_text.replace("```json", "").replace("```", "").strip()
        json_output = json.loads(generated_text)
      elif isinstance(response.text, str) and response.text.startswith("{"):
        json_output = json.loads(response.text)
        if json_output.get(k) is None or len(json_output.get(k))<1:
          continue
      elif isinstance(response.text, str):
        json_output = json.loads("{"+response.text+"}")
        if json_output.get(k) is None or len(json_output.get(k))<1:
          continue
      elif isinstance(response.text, dict):
        json_output = response.text
      else:
        print("Unknown response type")

      if json_output is not None:
        # print(json.dumps(json_output, indent=4))
        for key in json_output:
          if(len(json_output[key])>0):
            input["instruction"] = section_dict[k]
            full_json = {
                "input": input,
                "output": json_output
            }
            merged_json.append(full_json)
      time.sleep(60)

In [ ]:
refined_resume_section_file = "/content/eval_json_skill_ds.json"
with open(refined_resume_section_file, "w+", encoding='utf-8') as output_file:
  json.dump(merged_json, output_file, indent=4)